In today's puzzle we time travel (sic!) to the year 1518 where we need to avoid the guard. She, the guard, has a very organized way of moving and using those rules, we need to find the best way to avoid her. To read the full puzzle, check out the [AoC 2024 Day 6 puzzle](https://adventofcode.com/2024/day/6).

And here's how Chatty, with my instructions, envisions the 1518 elf world:



## Part 1: Mapping guards 

In [1]:
from pathlib import Path
import numpy as np

from typing import List, Tuple

tiny_example = """
..#.
#.^#
....
"""


def parse_input(input) -> Tuple[np.ndarray, Tuple[int, int], np.ndarray]:
    if Path(input).exists():
        with open(input, "r") as f:
            input = f.read()

    output_grid = []
    guard = ()

    guard_dir = {"^": [-1, 0], ">": [0, 1], "v": [1, 0], "<": [0, -1]}
    guard_dir = {k: np.array(v) for k, v in guard_dir.items()}

    for i, line in enumerate(input.strip().split("\n")):
        # check for guard
        output_grid.append([1 for _ in range(len(line))])
        for j, char in enumerate(line):
            if char not in [".", "#"]:
                if guard:
                    raise ValueError(
                        f"Multiple guards found at {guard} and {i, j}"
                    )
                guard = (i, j)
                guard_direction = guard_dir[char]
            elif char == "#":
                output_grid[i][j] = 0
    return np.array(output_grid), np.array(guard), guard_direction


parse_input(tiny_example)

(array([[1, 1, 0, 1],
        [0, 1, 1, 0],
        [1, 1, 1, 1]]),
 array([1, 2]),
 array([-1,  0]))

In [2]:
def guard_walk(
    grid: np.ndarray,
    guard_position: np.ndarray | list,
    guard_direction: np.ndarray | list,
):
    max_x, max_y = grid.shape
    visited = set()
    rot = np.array([[0, -1], [1, 0]])
    while True:
        visited.add(tuple(guard_position))
        # given the guard direction, check the next step
        next_step = guard_position + guard_direction
        # print(f"guards position: {guard_position}, next step: {next_step}")
        # check if next step is out of bounds
        if (
            next_step[0] < 0
            or next_step[0] >= max_x
            or next_step[1] < 0
            or next_step[1] >= max_y
        ):
            # finished walking
            return visited
        # print(f"next vaalue: {grid[next_step[0], next_step[1]]}")
        # check if next step is a wall
        if grid[next_step[0], next_step[1]] == 0:
            # rotate direction by 90 degrees right
            guard_direction = guard_direction @ rot

        else:
            # move to next step
            guard_position = next_step


len(guard_walk(*parse_input(tiny_example)))

2

In [3]:
len(guard_walk(*parse_input("../inputs/example_day_06.txt")))

41

In [4]:
%time len(guard_walk(*parse_input("../inputs/day_06.txt")))

CPU times: user 7.96 ms, sys: 825 μs, total: 8.79 ms
Wall time: 15.5 ms


4454

## Part 2: Trap the guard

In this task we need to trap the guard in a infinite loop. 



In [5]:
# if the guard visits start again with the same direction, that means it's a loop
# i need to keep track of the original direction and starting position

In [6]:
g, p, d = parse_input("../inputs/example_day_06.txt")

In [7]:
def print_grid(grid, guard=np.array([]), dr=np.array([])):
    d_ = {1: ".", 0: "X"}
    guard_dir = {"^": [-1, 0], ">": [0, 1], "v": [1, 0], "<": [0, -1]}
    rev_guard_dir = {tuple(v): k for k, v in guard_dir.items()}
    if len(guard) > 0:
        symb = rev_guard_dir[tuple(dr)]
    for i, row in enumerate(grid):
        row_ = [d_[i] for i in row]
        if symb and i == guard[0]:
            row_[guard[1]] = symb
        print("".join(row_))


print_grid(g, guard=p, dr=d)

....X.....
.........X
..........
..X.......
.......X..
..........
.X..^.....
........X.
X.........
......X...


In [27]:
def move(position, direction, visited, grid, obstacle_inserted=False):
    rot = np.array([[0, -1], [1, 0]])
    new_position = position + direction
    z = [
        np.array_equal(direction, d)
        for d in visited.get(tuple(new_position), [])
    ]
    if any(new_position >= grid.shape) or any(new_position < 0):
        return 0
    
    elif any(z):
        return 1

    elif grid[new_position[0], new_position[1]] == 0:
        return move(position, direction @ rot, visited, grid, obstacle_inserted)

    elif grid[new_position[0], new_position[1]] == 1 and obstacle_inserted:
        # already inserted an obstacle, so no inserting, just drifting
        return move(new_position, direction, visited, grid, obstacle_inserted)

    else:  # not inserted, so we will split between inserting and not inserting
        # if not inserted
        not_inserted_visited = visited.copy()

        if not any([np.array_equal(direction, d) for d in z]):
            z.append(direction)

        not_inserted_visited[(int(new_position[0]), int(new_position[1]))] = z

        new_grid = grid.copy()
        new_grid[new_position[0], new_position[1]] = 0
        # if inserted, then return the same, but insert obstacle at the grid next_position
        return move(
            new_position,
            direction,
            not_inserted_visited,
            grid,
            obstacle_inserted,
        ) + move(position, direction, visited, new_grid, obstacle_inserted=True)

In [28]:
move(p, d, {}, g)

6

In [13]:
def part_two(input):
    g, p, d = parse_input(input)
    return move(p, d, {}, g)

part_two("../inputs/example_day_06.txt")

6